In [44]:
import sys
import os

import pandas as pd
import geopandas as gpd

sys.path.append(r"C:\WBG\Work\Code\GOSTrocks\src")
import GOSTrocks.shapeMisc as shapeMisc
import GOSTrocks.dataMisc as dMisc
from GOSTrocks.misc import tPrint

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
# Define source data
FUA_file = r"C:\WBG\Work\data\URBAN\GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg"
OE_file = r"C:\WBG\Work\data\URBAN\OE_FUA_SHAPEFILE\OE_FUA_SHAPEFILE.shp"

out_folder = r"C:\WBG\Work\MENA_Urban\RESULTS\EXTENT_JOIN"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
inFUA = gpd.read_file(FUA_file)
inOE = gpd.read_file(OE_file)

inOE = inOE.to_crs(inFUA.crs)

In [46]:
inFUA.head()

,eFUA_ID,UC_num,UC_IDs,eFUA_name,Commuting,Cntry_ISO,Cntry_name,FUA_area,UC_area,FUA_p_2015,UC_p_2015,Com_p_2015,geometry
0,1.0,1.0,5959,Herat,1.0,AFG,Afghanistan,139.0,60.0,888811.175807,844574.564331,44236.611476,"MULTIPOLYGON (((5529000 4155000, 5531000 41550..."
1,192.0,1.0,5964,Guzarah,1.0,AFG,Afghanistan,32.0,16.0,169489.573231,160204.260864,9285.312366,"MULTIPOLYGON (((5541000 4139000, 5542000 41390..."
2,354.0,1.0,5968,Shindand,0.0,AFG,Afghanistan,12.0,12.0,147553.403870,147553.403870,0.000000,"MULTIPOLYGON (((5573000 4030000, 5574000 40300..."
3,505.0,1.0,5970,Qala i Naw,0.0,AFG,Afghanistan,3.0,3.0,79809.722656,79809.722656,0.000000,"MULTIPOLYGON (((5592000 4221000, 5590000 42210..."
4,648.0,1.0,5973,Farah,1.0,AFG,Afghanistan,32.0,16.0,131508.797060,122843.460327,8665.336733,"MULTIPOLYGON (((5607000 3923000, 5608000 39230..."


In [50]:
inOE.shape

(900, 5)

In [51]:
# convert inFUA geometry to centroid
inFUA['geometry'] = inFUA.centroid

# Spatial join inOE (centroids) to inFUA
joined_extents = gpd.sjoin_nearest(inOE.loc[:,['OE_FUAID', 'geometry']], inFUA.loc[:,['eFUA_ID', 'geometry']],  how='inner', distance_col='dist_to_fua')
#pd.DataFrame(joined_extents.drop(['geometry', 'index_right'], axis=1)).to_csv(os.path.join(out_folder, 'OE_FUA_join_raw.csv'))
joined_extents.sort_values(by='dist_to_fua', ascending=False)

,OE_FUAID,geometry,index_right,eFUA_ID,dist_to_fua
7,CPV99999,"MULTIPOLYGON Z (((-2305650.931 1845211.415 0, ...",854,38.0,262617.713387
800,USM71,"POLYGON Z ((-7377825.279 4164087.748 0, -73592...",8665,6809.0,59742.029603
500,SWZ99999,"POLYGON Z ((2916247.186 -3205661.463 0, 291899...",7960,330.0,20916.543186
53,DE037C,"POLYGON Z ((590122.279 5881989.516 0, 590738.9...",3237,3297.0,19005.932310
308,CANH_3,"POLYGON Z ((-6557943.927 5172097.359 0, -65560...",837,2428.0,16146.530109
...,...,...,...,...,...
314,CHN07004,"POLYGON Z ((9673980.676 4138971.968 0, 9673158...",1175,7004.0,0.000000
315,CHN10328,"MULTIPOLYGON Z (((11230950.659 3156211.824 0, ...",2432,10328.0,0.000000
316,CHN06640,"MULTIPOLYGON Z (((9645701.124 4778078.958 0, 9...",1123,6640.0,0.000000
317,CHN08057,"POLYGON Z ((10055356.866 4333879.288 0, 100510...",1496,8057.0,0.000000


In [41]:
joined_res = joined_extents.groupby('OE_FUAID').agg(count=('eFUA_ID', 'count'), FUA_IDs=('eFUA_ID', 'unique')).reset_index().sort_values('count', ascending=False)
joined_res.to_csv(os.path.join(out_folder, 'OE_GHSL_aggregated.csv'), index=False)